In [1]:
# Code attribution: Yiyin Shen, Tyler Caraza-Harter
# Imports
from flask import Flask, request, Response
from scipy.stats import fisher_exact
import matplotlib.pyplot as plt
import time
import pandas
import random

In [ ]:
# Build a webpage that does AB testing
app = Flask("app")

ab = [1, 1]
total = [2, 2]
cookie = "0"

@app.route("/")
def home():
    global ab, total, cookie
    if random.random() < 0.5:
        total[0] = total[0] + 1
        resp = Response('Cat! <a href="visit?from=cat">Visit</a>')
    else:
        total[1] = total[1] + 1
        resp = Response('Dog! <a href="visit?from=dog">Visit</a>')
    resp.set_cookie("visit", "0")
    return resp

@app.route("/visit")
def test():
    global ab, total, cookie
    cookie = request.cookies.get("visit", "1")
    if cookie == "0" and request.args:
        if request.args["from"] == "cat":
            ab[0] = ab[0] + 1
        else:
            ab[1] = ab[1] + 1
    df = pandas.DataFrame([[ab[0], total[0] - ab[0]], [ab[1], total[1] - ab[1]]], columns = ["click", "no click"], index = ["cat", "dog"])
    resp = Response("The p-value is " + str(fisher_exact(df).pvalue) + "<br>" + df.to_html())
    resp.set_cookie("visit", "1")
    return resp

app.run(host = "0.0.0.0", debug = False, threaded = False)

 * Serving Flask app 'app'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.140.197.58:5000
Press CTRL+C to quit
10.140.197.58 - - [23/Oct/2023 11:13:17] "GET / HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:13:28] "GET / HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:13:32] "GET / HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:13:41] "GET /visit?from=dog HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:13:50] "GET / HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:13:50] "GET / HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:13:55] "GET /visit?from=cat HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:17:39] "GET /visit?from=cat HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:17:40] "GET /visit?from=cat HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:17:55] "GET /visit?from=cat HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:24:13] "GET /visit?from=cat HTTP/1.1" 200 -
10.140.197.58 - - [23/Oct/2023 11:24:15] "GET /visit?from=cat HTTP/1.1" 200 -